이번 과제는 Bert Model을 사용하여 BBC 뉴스 기사의 category를 분류해보는 과제입니다. clone coding을 하시되, 코드 주석을 line by line으로 꼼꼼하게 달아보시며 공부해보세요!

## 데이터 로드 및 탐색

In [1]:
%%capture
!pip install transformers

In [2]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/data/NLP/bbc-text.csv')

In [5]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [6]:
print(len(df))

2225


In [7]:
df.groupby('category').count()

,text
category,
business,510
entertainment,386
politics,417
sport,511
tech,401


## BertTokenizer

토크나이저로 pretrain된 BERT의 BertTokenizer를 갖고 옵니다. 여러 종류를 시도해보세요.

- bert-base-uncased : 108MB param, all lowercase
- bert-large-cased : 340MB param, both upper and lower
- bert-base-cased : 108MB param, multi language, both upper and lower


In [26]:
# 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# 라벨링
labels = {'business':0,
          'entertainment':1,
          'sport':2,
          'tech':3,
          'politics':4
          }

## Dataset

In [27]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        # 카테고리 인코딩
        self.labels = [labels[label] for label in df['category']]
        # 토큰화
        # truncation = True (최대 길이를 초과하는 부분 잘라냄)
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    # 클래스 라벨 반환
    def classes(self):
        return self.labels

    # 데이터 길이 반환
    def __len__(self):
        return len(self.labels)

    # 특정 인덱스의 배치 라벨 반환
    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    # 특정 인덱스의 배치 텍스트 반환
    def get_batch_texts(self, idx):
        return self.texts[idx]

    # 특정 인덱스의 라벨과 텍스트 반환
    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

## Train & Evaluate BertClassifier

pretrain된 BertModel을 불러옵니다. 다른 간단한 층들도 같이 쌓아줍니다.

- bert-base-cased: 12-layer, 768-hidden, 12-self attention heads, 110M parameters. Trained on cased English text.


다른 종류들의 pretrianed model은 아래 링크에서 확인할 수 있습니다.

https://huggingface.co/transformers/v2.9.1/pretrained_models.html

In [28]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        # 사전 학습된 Bert모델 불러오기
        self.bert = BertModel.from_pretrained('bert-base-cased')
        # 드롭아웃 레이어 생성
        self.dropout = nn.Dropout(dropout)
        # 입력 768, 출력 5
        self.linear = nn.Linear(768, 5)
        # ReLU 함수 활성화
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        # input_id와 attention mask를 입력으로 받음
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        # 드롭아웃 적용
        dropout_output = self.dropout(pooled_output)
        # 선형레이어를 사용해 특징 변환
        linear_output = self.linear(dropout_output)
        # ReLU 함수 적용
        final_layer = self.relu(linear_output)

        # 최종 출력 반환
        return final_layer

In [29]:
def train(model, train_data, val_data, learning_rate, epochs):

    # 학습 데이터, 검증 데이터
    train, val = Dataset(train_data), Dataset(val_data)

    # 학습 데이터 로더
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    # 검증 데이터 로더
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    # CUDA 사용 가능 여부 확인
    use_cuda = torch.cuda.is_available()
    # 디바이스 지정
    device = torch.device("cuda" if use_cuda else "cpu")

    # 손실함수 - CrossEntropyLoss
    criterion = nn.CrossEntropyLoss()
    # 최적화 - Adam, 학습률
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            # 모델, 손실함수를 CUDA로 이동
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            # 학습 데이터 accuracy
            total_acc_train = 0
            # 학습 데이터 loss
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                # 학습 데이터 라벨, attention mask, input_id를 디바이스로 이동
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].squeeze(1).to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                # input_id, mask를 입력받아 출력 생성
                output = model(input_id, mask)

                # 배치 손실 계산
                batch_loss = criterion(output, train_label.long())
                # 총 손실 계산
                total_loss_train += batch_loss.item()

                # 정확도 계산
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                # 모델 기울기 초기화
                model.zero_grad()
                # 역전파 수행
                batch_loss.backward()
                # 최적화 수행
                optimizer.step()

            # 검증 데이터 accuracy
            total_acc_val = 0
            # 검증 데이터 loss
            total_loss_val = 0

            # 기울기 계산 비활성화
            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    # 검증 데이터 라벨, attention mask, input_ids 디바이스로 이동
                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].squeeze(1).to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    # input_id, mask를 입력받아 출력 생성
                    output = model(input_id, mask)

                    # 배치 손실, 총 손실 계산
                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    # 정확도 계산
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')


In [30]:
def evaluate(model, test_data):
    # 테스트 데이터
    test = Dataset(test_data)

    # 테스트 데이터 로더
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    # 디바이스 지정
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        # 모델 디바이스로 이동
        model = model.cuda()

    # 테스트 데이터 정확도
    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              # 테스트 데이터 라벨, attention mask, input_ids 디바이스로 이동
              test_label = test_label.to(device)
              mask = test_input['attention_mask'].squeeze(1).to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              # 출력
              output = model(input_id, mask)

              # 테스트 데이터 정확도
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [31]:
np.random.seed(112)
# 학습, 검증, 테스트 데이터 분리
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

1780 222 223


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [32]:
EPOCHS = 2 #EPOCH 수 늘려보기!
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

100%|██████████| 890/890 [03:08<00:00,  4.73it/s]


Epochs: 1 | Train Loss:  0.746 | Train Accuracy:  0.378 | Val Loss:  0.569 | Val Accuracy:  0.761


100%|██████████| 890/890 [03:14<00:00,  4.57it/s]


Epochs: 2 | Train Loss:  0.345 | Train Accuracy:  0.913 | Val Loss:  0.172 | Val Accuracy:  0.991


In [33]:
evaluate(model, df_test)

Test Accuracy:  0.978
